In [2]:
!pip install -q langchain numexpr sentencepiece plotly


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chat_models import ChatHuggingFace
import pandas as pd
from dotenv import load_dotenv
from scripts.evaluation import load_benchmark
import datasets
from huggingface_hub import login
import os

load_dotenv(override=True)
pd.set_option("max_colwidth", None)

In [5]:
login(os.getenv("HUGGINGFACEHUB_API_TOKEN"))

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/aymeric/.cache/huggingface/token
Login successful


In [6]:
from langchain.chat_models import ChatOpenAI
from transformers.agents import HfEngine
from transformers.agents.llm_engine import (
    get_clean_message_list,
    llama_role_conversions,
)

command_r_id = "CohereForAI/c4ai-command-r-plus"
zephyr_id = "HuggingFaceH4/zephyr-orpo-141b-A35b-v0.1"
mixtral_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llama3_8b_id = "meta-llama/Meta-Llama-3-8B-Instruct"
llama3_70b_id = "meta-llama/Meta-Llama-3-70B-Instruct"

# llm = HuggingFaceEndpoint(
#     repo_id=zephyr_id,
#     task="text-generation",
#     max_new_tokens=1024,
#     do_sample=False,
#     repetition_penalty=1.03,
# )

# llm_engine_hf = ChatHuggingFace(llm=llm)


class OAIEngine:
    def __init__(self):
        self.client = ChatOpenAI(model="gpt-4-1106-preview")

    def __call__(self, messages, stop=[]) -> str:
        # Get clean message list
        messages = get_clean_message_list(
            messages, role_conversions=llama_role_conversions
        )

        return self.client.invoke(messages, stop=stop).content


eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
eval_model_name = "GPT4"


# def call_llm_hf(input: str, stop=["Observation", "Final Answer"]) -> str:
#     return llm_engine_hf.invoke(input, stop=stop).content


# call_llm_hf("Please output 'Observation'")

/Users/aymeric/venvs/ml/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
OPENAI_MODEL_ID = "gpt-4-0125-preview"
OUTPUT_DIR = "output_reasoning"

In [8]:
eval_df = load_benchmark()
eval_df["true_answer"] = eval_df["true_answer"].astype(str)
eval_ds = datasets.Dataset.from_pandas(eval_df)

### Define tools

In [9]:
from transformers.agents.default_tools import CalculatorTool, Tool
from langchain_community.utilities import SerpAPIWrapper

params = {
    "engine": "bing",
    "gl": "us",
    "hl": "en",
}
langchain_serpapi = SerpAPIWrapper(params=params)


class SearchTool(Tool):
    name = "search"
    description = "A search engine. Useful for when you need to answer questions about current events. Input should be a search query."

    inputs = {"query": {"description": "your search query", "type": str}}
    output_type = str

    def __call__(self, query: str) -> str:
        return langchain_serpapi.run(query)

In [10]:
TOOLBOX_CODE = [SearchTool()]

TOOLBOX_JSON = [SearchTool(), CalculatorTool()]

# Define agents

In [11]:
from transformers import CodeAgent, ReactCodeAgent, ReactJSONAgent

from scripts.agents import build_hf_agent_with_tools, build_openai_agent_with_tools


react_agent_openai = ReactCodeAgent(
    llm_engine=OAIEngine(), tools=TOOLBOX_CODE, max_iterations=7
)

# langchain_agent = build_hf_agent_with_tools(
#     repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1"
# )
# langchain_openai_agent = build_openai_agent_with_tools()

react_json_agent_llama3_70 = ReactJSONAgent(
    llm_engine=HfEngine(model=llama3_70b_id),
    tools=TOOLBOX_JSON,
    max_iterations=7,
)

react_code_agent_llama3_70 = ReactCodeAgent(
    llm_engine=HfEngine(model=llama3_70b_id),
    tools=TOOLBOX_CODE,
    max_iterations=7,
)

react_code_agent_llama3_8 = ReactCodeAgent(
    llm_engine=HfEngine(model=llama3_8b_id),
    tools=TOOLBOX_CODE,
    max_iterations=7,
)

react_code_agent_mixtral_8x7 = ReactCodeAgent(
    llm_engine=HfEngine(model=mixtral_id),
    tools=TOOLBOX_CODE,
    max_iterations=7,
)

react_json_agent_mixtral_8x7 = ReactJSONAgent(
    llm_engine=HfEngine(model=mixtral_id),
    tools=TOOLBOX_JSON,
    max_iterations=7,
)

agents = {
    "react_code_llama3-70b_23-04": react_code_agent_llama3_70,
    "react_code_mixtral_8x7_23-04": react_code_agent_mixtral_8x7,
    "react_json_mixtral_8x7_24-04": react_json_agent_mixtral_8x7,
    "react_code_gpt4_23-04": react_agent_openai,
    "react_json_llama3-70b_23-04": react_json_agent_llama3_70,
    "react_code_llama3-8b_24-04": react_code_agent_llama3_8,
}


No tool description template is defined for this tokenizer - using a default tool description template that implements the ChatML format (without BOS/EOS tokens!). If the default is not appropriate for your model, please set `tokenizer.tool_description_template` to an appropriate template. 


In [12]:
async def call_transformers_agent(agent, question: str) -> str:
    result = agent.run(question)
    return {
        "output": str(result),
        "intermediate_steps": agent.logs.copy(),
    }


async def call_langchain(agent, question: str) -> str:
    output = agent.invoke({"input": question})
    return {
        "output": output["output"],
        "intermediate_steps": [el[0].log for el in output["intermediate_steps"]],
    }


async def call_vanilla_llm(agent, question: str) -> str:
    result = agent(question)
    return {
        "output": str(result),
        "intermediate_steps": [],
    }


# await call_langchain(langchain_agent, "Please provide a final answer of '7'")

In [12]:
res = await call_transformers_agent(react_agent_openai, "How much is 3 power 0.57?")

1.8705060412286219


In [13]:
agents

{'react_code_llama3-70b_23-04': <transformers.agents.agents.ReactCodeAgent at 0x2d0d5d8d0>,
 'react_code_mixtral_8x7_23-04': <transformers.agents.agents.ReactCodeAgent at 0x2d0d5e650>,
 'react_json_mixtral_8x7_24-04': <transformers.agents.agents.ReactJSONAgent at 0x2d0d5ea10>,
 'react_code_gpt4_23-04': <transformers.agents.agents.ReactCodeAgent at 0x2cf2e5f10>,
 'react_json_llama3-70b_23-04': <transformers.agents.agents.ReactJSONAgent at 0x2d0b9d410>,
 'react_code_llama3-8b_24-04': <transformers.agents.agents.ReactCodeAgent at 0x2d0d5dc50>}

In [14]:
assert False

AssertionError: 

In [16]:
from scripts.run_agents import run_full_tests

results = await run_full_tests(
    eval_ds,
    agents,
    output_folder=OUTPUT_DIR,
    agent_call_function=call_transformers_agent,
    key_for_answer="true_answer",
)

Found 90 previous results!


100%|██████████| 90/90 [00:00<00:00, 10574.76it/s]
/Users/aymeric/Documents/Code/agent_reasoning_benchmark/scripts/run_agents.py:232: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  results = pd.read_json(output_path, lines=True).to_dict(orient="records")


Expected object or value
Found no usable records! 🤔 Starting new.


  1%|          | 1/90 [00:11<16:55, 11.41s/it]Error in code parsing: The code blob you used is invalid: due to the following error: 'NoneType' object has no attribute 'group'. This means that the regex pattern ```(?:py)?\n(.*?)``` was not respected. Make sure to correct its formatting. Code blob was: 
```py
# Determine the number of cats
cats = x . Be sure to provide correct code
Error in code parsing: The code blob you used is invalid: due to the following error: 'NoneType' object has no attribute 'group'. This means that the regex pattern ```(?:py)?\n(.*?)``` was not respected. Make sure to correct its formatting. Code blob was: 
```py
# Determine the number of cats
cats = x . Be sure to provide correct code
25
  2%|▏         | 2/90 [00:30<23:09, 15.79s/it]Error in code parsing: The code blob you used is invalid: due to the following error: 'NoneType' object has no attribute 'group'. This means that the regex pattern ```(?:py)?\n(.*?)``` was not respected. Make sure to correct its fo

The code generated by the agent is not valid.
 invalid syntax (<unknown>, line 2)


5
 23%|██▎       | 21/90 [05:09<17:21, 15.10s/it]list index out of range
Traceback (most recent call last):
  File "/Users/aymeric/Documents/Code/transformers_agents2/transformers_agents/src/transformers/agents/agents.py", line 281, in extract_action
    split[-2],
    ~~~~~^^^^
IndexError: list index out of range
Error: No 'Code:' token provided in your output.
Your output:
 
. Be sure to include an action, prefaced with 'Code:'!
list index out of range
Traceback (most recent call last):
  File "/Users/aymeric/Documents/Code/transformers_agents2/transformers_agents/src/transformers/agents/agents.py", line 281, in extract_action
    split[-2],
    ~~~~~^^^^
IndexError: list index out of range
Error: No 'Code:' token provided in your output.
Your output:
 Ticket cost: 20
Combo meal cost: 11
Candy cost: 5.0

```java

```

ahuman
Connor will spend a total of $36.0 on his date.

Thought: I have calculated the total cost for Connor's date. According to the observation, the ticket cost is $2

# Evaluate

In [13]:
import glob

answer_file_path = f"{OUTPUT_DIR}/answers.jsonl"

result_df = pd.concat(
    [pd.read_json(f, lines=True) for f in glob.glob(f"{OUTPUT_DIR}/*.jsonl")]
)
result_df = result_df.drop(columns=["start_time", "end_time"])
result_df.to_json(answer_file_path, lines=True, orient="records")

### Exact match for GSM8K

In [14]:
from scripts.evaluation import score_any_match

results_math = result_df.loc[result_df["task"] == "gsm8k"].copy()
results_math["exact_match"] = -1
results_math["exact_match"] = results_math.apply(
    lambda row: score_any_match(row["prediction"], float(row["true_answer"])), axis=1
)

Error when extracting string: could not convert string to float: '64G'
Error when extracting string: could not convert string to float: '19t'
Error when extracting string: expected string or bytes-like object, got 'NoneType'
Error when extracting string: expected string or bytes-like object, got 'NoneType'
Error when extracting string: expected string or bytes-like object, got 'NoneType'
Error when extracting string: expected string or bytes-like object, got 'NoneType'
Error when extracting string: expected string or bytes-like object, got 'NoneType'
Error when extracting string: expected string or bytes-like object, got 'NoneType'
Error when extracting string: expected string or bytes-like object, got 'NoneType'
Error when extracting string: expected string or bytes-like object, got 'NoneType'
Error when extracting string: expected string or bytes-like object, got 'NoneType'
Error when extracting string: expected string or bytes-like object, got 'NoneType'
Error when extracting string

In [15]:
results_math.groupby(["agent_name", "task"])["exact_match"].mean()

agent_name                    task 
react_code_gpt4_23-04         gsm8k    0.900000
react_code_llama3-70b_23-04   gsm8k    0.950000
react_code_llama3-8b_24-04    gsm8k    0.262500
react_code_mixtral_8x7_23-04  gsm8k    0.575000
react_json_llama3-70b_23-04   gsm8k    0.925000
react_json_mixtral_8x7_24-04  gsm8k    0.508333
Name: exact_match, dtype: float64

### LLM judge for others

In [ ]:
answers_nonmath = result_df.loc[result_df["task"] != "gsm8k"].to_dict(orient="records")

In [ ]:
from langchain.chat_models import ChatOpenAI

eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
eval_model_name = "GPT4"

In [ ]:
# evaluations = pd.read_json('output_reasoning/evaluation.jsonl', lines=True)
# print(len(evaluations))
# evaluations['agent_name'].unique()

# evaluations = evaluations.loc[~evaluations["agent_name"].str.contains("8b")]
# print(len(evaluations))

# evaluations.to_json('output_reasoning/evaluation.jsonl', lines=True, orient='records')

230


array(['react_code_gpt4_23-04', 'react_json_llama3-70b_23-04',
       'react_code_llama3-70b_23-04', 'react_code_mixtral_8x7_23-04',
       'react_json_mixtral_8x7_24-04'], dtype=object)

In [ ]:
from scripts.evaluation import evaluate_answers
from scripts.prompts import EVALUATION_PROMPT_TEMPLATE

output_file_path = f"{OUTPUT_DIR}/evaluation.jsonl"

run_evaluation = True
if run_evaluation:
    evaluated_answers_nonmath = await evaluate_answers(
        answers_nonmath,
        eval_chat_model,
        "GPT4",
        EVALUATION_PROMPT_TEMPLATE,
        output_file_path=output_file_path,
    )
    print("Evaluation is complete!")

Previous evaluations:
                       agent_name  \
0           react_code_gpt4_23-04   
1     react_json_llama3-70b_23-04   
2     react_code_llama3-70b_23-04   
3     react_code_llama3-70b_23-04   
4     react_code_llama3-70b_23-04   
..                            ...   
179   react_json_llama3-70b_23-04   
180  react_code_mixtral_8x7_23-04   
181  react_code_mixtral_8x7_23-04   
182   react_json_llama3-70b_23-04   
183   react_json_llama3-70b_23-04   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

  0%|          | 0/146 [00:00<?, ?it/s]

Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating e

100%|██████████| 146/146 [00:51<00:00,  2.81it/s]

Evaluation is complete!


In [ ]:
df = pd.DataFrame(evaluated_answers_nonmath)
df.loc[(df["agent_name"] == "react_code_llama3-8b_24-04")].head(10)

agent_name  \
0   react_code_llama3-8b_24-04   
3   react_code_llama3-8b_24-04   
4   react_code_llama3-8b_24-04   
5   react_code_llama3-8b_24-04   
6   react_code_llama3-8b_24-04   
8   react_code_llama3-8b_24-04   
10  react_code_llama3-8b_24-04   
11  react_code_llama3-8b_24-04   
12  react_code_llama3-8b_24-04   
13  react_code_llama3-8b_24-04   

                                                                                                                                                                                                                                                                                                                  question  \
0                                                                                                                                                                                                                                     Which school was founded first, Kasetsart University or Holy Cross of Davao College?   
3                                                                                                                                                                                                                           What act of a mythical hunter-warrior of Irish mythology won him the leadership of the Fianna?   
4                                                What was the complete title of the book in which two James Beard Award winners recommended the restaurant where Ali Khan enjoyed a New Mexican staple in his cost-conscious TV show that started in 2015? Write the numbers in plain text if there are some in the title.   
5                                                                                                                                                                                                                                     Which type of dog is the national dog breed of Israel: Canaan Dog or Dutch Shepherd?   
6   Roxanne Quimby, is an American artist, conservationist and businesswoman notable for founding which North Carolina-based personal care products company with the eponymous beekeeper, Ingram Berg "Burt" Shavitz, was an American beekeeper and businessman notable for founding which personal care products company?   
8                                                                                                                                                                                                                                            Which musician filled more roles in their bands, Vibeke Stene or Wong Ka Kui?   
10                                                                                                                                                                                                                                                   Viken Babikian and John Najarian, both work as which mutual location?   
11                                                                                                                                                                                                                                                   Where are the headquarters of the league the Denver Pioneers play in?   
12                                                                                                                                                                                                                                    Which type of dog is the national dog breed of Israel: Canaan Dog or Dutch Shepherd?   
13                                                                                                         What is the other name for the world's largest dōjinshi fair, held twice a year in Tokyo, Japan that was the location of the first release of a Japanese adult dōjin visual novel created by Type-Moon in 2000?   

                                             prediction  \
0                                  Kasetsart University   
3                           

In [ ]:
results_nonmath = pd.DataFrame.from_dict(evaluated_answers_nonmath)


def interpret_score(eval_score):
    try:
        return (float(eval_score) - 1) / 4
    except:
        return 0


results_nonmath["eval_score_GPT4"] = results_nonmath["eval_score_GPT4"].apply(
    interpret_score
)
results_nonmath.loc[results_nonmath["task"] == "GAIA"].groupby("agent_name")[
    "eval_score_GPT4"
].mean()

agent_name
react_code_gpt4_23-04           0.512500
react_code_llama3-70b_23-04     0.323529
react_code_llama3-8b_24-04      0.025000
react_code_mixtral_8x7_23-04    0.000000
react_json_llama3-70b_23-04     0.409091
Name: eval_score_GPT4, dtype: float64

### Aggregate evaluations

In [ ]:
result_df = pd.concat([results_math, results_nonmath])
result_df["aggregate_score"] = (
    result_df["exact_match"].fillna(0) + result_df["eval_score_GPT4"].fillna(0)
) * 100
result_df.groupby(["agent_name", "task"])["aggregate_score"].mean()

agent_name                    task    
react_code_gpt4_23-04         GAIA            51.25
                              HotpotQA    60.833333
                              gsm8k            90.0
react_code_llama3-70b_23-04   GAIA        32.352941
                              HotpotQA    65.833333
                              gsm8k            95.0
react_code_llama3-8b_24-04    GAIA              2.5
                              HotpotQA        23.75
                              gsm8k           26.25
react_code_mixtral_8x7_23-04  GAIA              0.0
                              HotpotQA         42.5
                              gsm8k            57.5
react_json_llama3-70b_23-04   GAIA        40.909091
                              HotpotQA    71.666667
                              gsm8k            92.5
react_json_mixtral_8x7_24-04  HotpotQA    76.785714
                              gsm8k            45.0
Name: aggregate_score, dtype: object

In [ ]:
list_correct_langchain = result_df.loc[
    (result_df["agent_name"] == "langchain_gpt-4")
    & (result_df["task"] == "GAIA")
    & (result_df["aggregate_score"] >= 50),
    "question",
].unique()
list_wrong_transformers = result_df.loc[
    (result_df["agent_name"] == "react_text_gpt-4")
    & (result_df["task"] == "GAIA")
    & (result_df["aggregate_score"] < 70),
    "question",
].unique()

intersection = [el for el in list_correct_langchain if el in list_wrong_transformers]

### Display

In [ ]:
result_df["intermediate_steps"].values[1][5]

IndexError: list index out of range

In [ ]:
# result_df.loc[result_df["intermediate_steps"].apply(lambda x: "error" in str(x))]

In [ ]:
import plotly.express as px

aggregate = (
    result_df.groupby(["agent_name", "task"])[["aggregate_score"]].mean().reset_index()
)
# aggregate["agent_name"] = aggregate["agent_name"].map(
#     {
#         # "vanilla_llm": "LLM",
#         # "react": "ReactAgent (Mixtral - messages)",
#         # "code": "CodeAgent",
#         # "react_prev": "ReactAgent (Mixtral - text)",
#         # "react_messages_chat_model": "ReactAgent (Nous - messages)",
#         # "react_text_chat_model": "ReactAgent (Nous - text)",
#         # "react_messages_openai": "ReactAgent (OpenAI - messages)",
#         "react_text_openai": "OpenAI: GPT3.5",
#         # "react_text_mixtral2": "ReactAgent (Mixtral2 - text)",
#         # "react_text_mixtral3": "ReactAgent (Mixtral3 - text)",
#         # "react_text_mixtral4": "Mixtral (Ours)",
#         # "react_text_mixtral-15-04": "Mixtral-8x7b",
#         "react_messages_llama3-19-04": "OS: Llama3-70B-Instruct",
#         "react_text_llama3-8b-19-04": "OS: Llama3-8B-Instruct",
#         # "react_text_mixtral_nojson": "ReactAgent (Mixtral4 - nojson)",
#         # "langchain_agent": "Langchain",
#         "react_text_command-r-plus": "OS: Command-R-Plus",
#         "react_text_llama3-70b-19-04-2": "OS: Llama3-70B-Instruct Code",
#         # "react_text_mixtral_huge": "Mixtral-8x22B",
#         "react_text_gpt-4": "OpenAI: GPT4",
#         # "langchain_gpt-4": "GPT4 (LangChain)",
#     }
# )
aggregate = aggregate.sort_values("agent_name", ascending=True)
fig = px.bar(
    aggregate,
    x="agent_name",
    y="aggregate_score",
    color="task",
    labels={
        "agent_name": "<b>Agent</b>",
        "task": "<b>Task</b>",
        "aggregate_score": "Performance",
        "eval_score_GPT4": "<b>Score</b>",
    },
)
fig.update_layout(
    width=len(aggregate["agent_name"].unique()) * 100 + 200,
    height=600,
    barmode="group",
    bargap=0.35,
    bargroupgap=0.0,
    yaxis_range=[0, 105],
)
fig.update_traces(texttemplate="%{y:.0f}", textposition="outside")
fig.layout.yaxis.ticksuffix = "%"
fig.show()

/Users/aymeric/venvs/ml/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])
